# Content:

- parse IFC models into graph database
- run Diff
- generate patch
- apply patch

## Parse BIM models into graph database

In [28]:
import time
from typing import List

from IfcGraphInterface.Ifc2GraphTranslator import IFCGraphGenerator
from neo4j_middleware.neo4jConnector import Neo4jConnector

print('[INFO] Parsing Ifc StepP21 model to Neo4j.... \n')

# connect to neo4j
connector = Neo4jConnector()
connector.connect_driver()

# paths to IFC models
model_paths: List[str] = [
    '00_sampleData/IFC_stepP21/diss_samples/cube_single.ifc',
    '00_sampleData/IFC_stepP21/diss_samples/cube_single-PosChange.ifc']

print('[INFO] Starting to generate graphs...')
amount = len(model_paths)
start = time.perf_counter()

model_labels = []

for idx, path in enumerate(model_paths):
    # parse model
    # ToDo fix parsing issue
    graphGenerator = IFCGraphGenerator(connector, path, write_to_file=False)
    print('Generating Graph %d/%d' % (idx + 1, amount))
    label = graphGenerator.generateGraph()
    model_labels.append(label)

finish = time.perf_counter()
print('\n[INFO] 100% done. Graphs generated. Finished in {} seconds.'.format(round(finish - start, 2)))
# disconnect from database
connector.disconnect_driver()

label_init = model_labels[0]
label_updated = model_labels[1]

del idx, finish, start, model_labels, label, graphGenerator

[INFO] Parsing Ifc StepP21 model to Neo4j.... 

[INFO] Starting to generate graphs...
Generating Graph 1/2
[IFC_P21 > ts20210623T000000 < ]: Generating graph... 
                                                                                                              
[IFC_P21 > ts20210623T000000 < ]: Generating graph - DONE. 
 
Generating Graph 2/2
[IFC_P21 > ts20210623T000006 < ]: Generating graph... 
                                                                                                              
[IFC_P21 > ts20210623T000006 < ]: Generating graph - DONE. 
 

[INFO] 100% done. Graphs generated. Finished in 2.99 seconds.


## Run diff

runs the depth-first traversal to compare both model graphs and connects equivalent nodes using `EQUIVALENT_TO` edges

In [32]:
from neo4jGraphDiff.GraphDiff import GraphDiff
from neo4j_middleware.ResponseParser.NodeItem import NodeItem
from neo4j_middleware.neo4jConnector import Neo4jConnector
import jsonpickle

print('Run Diff ... \n')

label_init = "ts20210623T000000"
label_updated = "ts20210623T000004"

# connect to neo4j
connector = Neo4jConnector()
connector.connect_driver()

# get topmost entry nodes
raw_init = connector.run_cypher_statement(
    """
    MATCH (n:PrimaryNode:{} {{EntityType: "IfcProject"}})
    RETURN n
    """.format(label_init))
raw_updated = connector.run_cypher_statement(
    """
    MATCH (n:PrimaryNode:{} {{EntityType: "IfcProject"}})
    RETURN n
    """.format(label_updated))

entry_init: NodeItem = NodeItem.from_neo4j_response(raw_init)[0]
entry_updated: NodeItem = NodeItem.from_neo4j_response(raw_updated)[0]

# run diff
pDiff = GraphDiff(connector=connector, ts_init=label_init, ts_updated=label_updated)
delta = pDiff.diff_subgraphs(entry_init, entry_updated)

# Create EQUIVALENT_TO relationships to mark all nodePairs that are matched
print('[INFO] building EQUIVALENT_TO edges ... ')
pDiff.build_equivalent_to_edges()
print('[INFO] building EQUIVALENT_TO edges: DONE. ')

# store result
print('[INFO] saving delta ... ')
f = open('GraphDelta_init{}-updt{}.json'.format(label_init, label_updated), 'w')
f.write(jsonpickle.dumps(delta))
f.close()
print('[INFO] saving delta: DONE. ')


connector.disconnect_driver()
del raw_init, raw_updated, entry_init, entry_updated, GraphDiff, f

Run Diff ... 


IndexError: list index out of range

## Generate patch

In [31]:
from PatchManager.GraphPatchService import GraphPatchService
from neo4j_middleware.neo4jConnector import Neo4jConnector

connector = Neo4jConnector()
connector.connect_driver()

service = GraphPatchService.from_existing_delta(delta)
# service.load_delta('GraphDelta_init{}-updt{}.json'.format(label_init, label_updated))

patch = service.generate_patch()

service.save_patch_to_json(patch)

# visualize results
# update_patch.operations[0].plot_patterns()

# finally disconnect
connector.disconnect_driver()


AttributeError: 'NoneType' object has no attribute 'paths'

# Show patch patterns

In [27]:
print("Topo Changes:")

# for op in patch.operations:
#     print("TYPE: {}".format(op.operation_type))
#     print("Glue")
#     print(op.gluing_pattern.print_to_console())
#     print("Context")
#     print(op.context_pattern.print_to_console())
#     print("PushOut")
#     print(op.push_out_pattern.print_to_console())
#     print()
    
print("Semantic Changes:")
for op in patch.attribute_changes: 
    print("path")
    print(op.path.to_cypher(skip_timestamp=True, entType_guid_only=True))
    print("KEY: {}".format(op.attribute_name))
    print("VAL_OLD: {}".format(op.init_value))
    
    print("VAL_NEW: {}".format(op.updated_value))

Topo Changes:
Semantic Changes:
path
pathNone = (n98:IfcBuildingElementProxy:PrimaryNode{EntityType:"IfcBuildingElementProxy", GlobalId:"3xn$Io5tr8TOYxqxH15Rkg"})-[e21:rel{rel_type:"ObjectPlacement"}]->(n99:IfcLocalPlacement:SecondaryNode{EntityType:"IfcLocalPlacement"}) -[e24:rel{rel_type:"RelativePlacement"}]->(n100:SecondaryNode:IfcAxis2Placement3D{EntityType:"IfcAxis2Placement3D"}) -[e25:rel{rel_type:"Location"}]->(n101:IfcCartesianPoint:SecondaryNode{EntityType:"IfcCartesianPoint"})
KEY: Coordinates
VAL_OLD: (2.0, 5.0, 1.0)
VAL_NEW: (1.0, 7.0, 1.0)


## Apply patch

! Switch to receiver database !

optional: clear database

In [ ]:
from neo4j_middleware.neo4jConnector import Neo4jConnector

# connect to neo4j
connector = Neo4jConnector()
connector.connect_driver()
print("[INFO] clear database ... ")
connector.run_cypher_statement("MATCH (n) DETACH DELETE n")
connector.disconnect_driver()
print("[INFO] clear database: DONE. ")

optional: load initial model

In [ ]:
from IfcGraphInterface.Ifc2GraphTranslator import IFCGraphGenerator
from neo4j_middleware.neo4jConnector import Neo4jConnector

# connect to neo4j
connector = Neo4jConnector()
connector.connect_driver()

# paths to IFC models
model_paths: List[str] = [
    './00_sampleData/IFC_stepP21/Fahrzeughalle/Fahrzeughalle.ifc',
    './00_sampleData/IFC_stepP21/Fahrzeughalle/Fahrzeughalle_updt.ifc']

graphGenerator = IFCGraphGenerator(connector, model_paths[0], None)
print('[INFO] Generating host graph... ' )
ts_host = graphGenerator.generateGraph()
print('[INFO] Generating host graph: DONE. ')
# finally disconnect
connector.disconnect_driver()

Apply patch

In [ ]:
from PatchManager.PatchService import PatchService
from neo4j_middleware.neo4jConnector import Neo4jConnector

label_init = "ts20220510T140147"
label_updated = "ts20220510T140041"

# connect to neo4j
connector = Neo4jConnector()
connector.connect_driver()

# init new PatchService object handling all load and save operations
service = PatchService()

# load patch
patch = service.load_patch_from_json('Patch_init{}-updt{}.json'.format(label_init, label_updated))

# apply the patch
print('[INFO] Applying patch ...' )
service.apply_patch(patch)
print('[INFO] Applying patch: DONE.' )

# finally disconnect
connector.disconnect_driver()

In [ ]:
# harmonize labels
label_init = "ts20210623T091748"
label_updated = "ts20210623T091749"

connector.run_cypher_statement("MATCH (n) REMOVE n:{} SET n:{}".format(label_init, label_updated))

# Parse graph back into file-based representation

In [ ]:
from IfcGraphInterface.Graph2IfcTranslator import Graph2IfcTranslator
from neo4j_middleware.neo4jConnector import Neo4jConnector

# connect to neo4j
connector = Neo4jConnector()
connector.connect_driver()

label_updated = "ts20220510T140147"

print('[INFO] Parsing graph to Ifc StepP21 model... ')
generator = Graph2IfcTranslator(connector=connector, ts=label_updated)
generator.generate_SPF()

path = "C:\dev\out\{}".format(label_updated)
generator.save_model(path=path)

print('[INFO] Parsing graph to Ifc StepP21 model: DONE. ')
print('[INFO] path: {}'.format(path))

# finally disconnect
connector.disconnect_driver()

inverse patch

In [ ]:
from neo4j_middleware.neo4jConnector import Neo4jConnector
from PatchManager.PatchService import PatchService

# connect to neo4j
connector = Neo4jConnector()
connector.connect_driver()

label_init = "ts20210623T091748"
label_updated = "ts20210623T091749"

# init new PatchService object handling all load and save operations
service = PatchService()

# load patch
patch = service.load_patch_from_json('Patch_init{}-updt{}.json'.format(label_init, label_updated))

# apply the patch
service.apply_patch_inverse(patch, connector=connector)

connector.disconnect_driver()